In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.
api_key = os.getenv("OPENAI_API_KEY")
# Step 1: Define the prompt
# prompt = PromptTemplate.from_template("Translating the sentence: {sentence} into any language of your choice.")
# prompt = PromptTemplate.from_template("Rewrting the sentence more professionally: {sentence}")

prompt = PromptTemplate.from_template("Answer the following question clearly and concisely:\n\n{question}")

# Step 2: Define the LLM
llm = OpenAI(
    temperature=0.7,
    openai_api_key=api_key
    )

# Step 3: Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

# Step 4: Run it
user_question = input("Enter your question: ")
result = chain.run(question=user_question)
print(result)




I am an AI and do not have the ability to feel emotions. I am functioning as expected and ready to assist you with any tasks you need help with. Thank you for asking.


In [17]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Global vectorstore to store indexed documents
vectorstore = None

def upload_pdf(file_path):
    global vectorstore

    try:
        # Load the PDF
        loader = PyPDFLoader(file_path)
        documents = loader.load()

        # Split the text into manageable chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_documents(documents)

        # Generate embeddings
        embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
        vectorstore = FAISS.from_documents(chunks, embeddings)

        retriever = vectorstore.as_retriever()
        llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
        qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

        return "✅ PDF uploaded and indexed successfully!"
    except Exception as e:
        return f"❌ Error: {str(e)}"
    
def ask_question(query):
    if qa_chain is None:
        return "❗ Please upload a PDF first."
    try:
        answer = qa_chain.run(query)
        return answer
    except Exception as e:
        return f"❌ Error: {str(e)}"
    # Its a general Chatbot in 

chat_history = []  # list of (user, bot) tuples
def general_chatbot_response(user_input):
    try:
        prompt = f"You are a helpful assistant. Answer the following:\n\n{user_input}"
        response = llm.invoke(prompt)
        # Store Q&A in history
        chat_history.append(("You: " + user_input, "Bot: " + str(response)))
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}"
    

def get_chat_history():
    if not chat_history:
        return "No chat history yet."
    return "\n\n".join([f"{q}\n{a}" for q, a in chat_history])


In [20]:
# -------- Gradio UI --------
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Chatbot App (PDF Q&A + General Assistant)")

    with gr.Tab("📄 Chat with PDF"):
        with gr.Row():
            file_input = gr.File(label="Upload PDF", type="filepath", file_types=[".pdf"])
            upload_output = gr.Textbox(label="Upload Status", lines=1)
        file_input.change(upload_pdf, inputs=file_input, outputs=upload_output)

        with gr.Row():
            pdf_question = gr.Textbox(label="Ask a question about the PDF")
            pdf_answer = gr.Textbox(label="Answer", lines=5)
        pdf_question.submit(general_chatbot_response, inputs=pdf_question, outputs=pdf_answer)

    with gr.Tab("💬 General Q&A Chatbot"):
        with gr.Row():
            user_question = gr.Textbox(label="Ask anything")
            general_answer = gr.Textbox(label="Bot Answer", lines=5)
        user_question.submit(general_chatbot_response, inputs=user_question, outputs=general_answer)
    with gr.Row():
        show_history_btn = gr.Button("📜 Show Chat History")
        history_output = gr.Textbox(label="Chat History", lines=10)
        user_question.submit(general_chatbot_response, inputs=user_question, outputs=general_answer)
        show_history_btn.click(get_chat_history, inputs=[], outputs=history_output)



demo.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
